In [61]:
date = "2024-01-03"  # to get csv

In [62]:
import importlib
from pathlib import Path

from modules import module
import numpy as np
import pandas as pd

try:
    importlib.reload(module)  # reload module
except NameError:
    pass

# Load DF

In [63]:
path = Path(f"input/{date}.csv")
df = pd.read_csv(path)

target_cols = [
    "External ID",
    "Display Name",
    "Date of Birth",
    "Email",
    "Phone",
    "Contract Type",
    "From",
    "To",
    "Fully Paid Date",
    "Membership/Display Name",
    "Membership Status",
    "Partner/Branch/Display Name",
    "Partner/Date of Birth",
    "Partner/Age",
    "Partner/City",
    "Partner/Gender",
    "Partner/Household income/month",
    "Partner/Industry/Display Name",
    "Partner/Job Position",
    "Partner/Occupation",
    "Partner/Street",
    "Partner/Street2",
    "Followers (Partners)/Hobby",
    "Followers (Partners)/Interest",
    "Partner/Hobby",
    "Partner/Interest",
]
current_cols = list(df.columns)
map_cols = dict(zip(current_cols, target_cols))
df.columns = df.columns.map(map_cols)

print(df.shape)
df.sample(3)

(29338, 26)


,External ID,Display Name,Date of Birth,Email,Phone,Contract Type,From,To,Fully Paid Date,Membership/Display Name,...,Partner/Household income/month,Partner/Industry/Display Name,Partner/Job Position,Partner/Occupation,Partner/Street,Partner/Street2,Followers (Partners)/Hobby,Followers (Partners)/Interest,Partner/Hobby,Partner/Interest
7131,__export__.membership_membership_line_191401_d...,ALI JUPRI,1977-11-01,alijupri2012@gmail.com,081316368197,Private,2021-03-16,2023-03-15,2021-02-22,[CLB.DLX.24] 24 Months Membership DLX - DG,...,NaN,NaN,NaN,Narcotics Division,JL. MULIA NO. 15 BUMI PITALOKA,NaN,NaN,NaN,Hiking,Sports
15855,__export__.membership_membership_line_186832_6...,PARK SEUNG CHAN,NaN,NaN,NaN,NaN,2019-07-01,2020-06-01,2020-03-03,[CLB.DLX.9] 9 Months Membership DLX - PP,...,NaN,NaN,OTHERS,NaN,JAKARTA,NaN,NaN,NaN,Reading,Science
15880,__export__.membership_membership_line_160485_5...,RINA SANDRIA,NaN,NaN,NaN,NaN,2019-03-31,2020-05-31,NaN,NaN,...,NaN,NaN,NaN,NaN,Perum Jan Ang Ratmil Blok H 17,NaN,NaN,NaN,NaN,NaN


# Clean DF

In [64]:
df_clean = (
    df
    # drop na
    .dropna(subset="From")
    .dropna(subset="To")
    # drop date too old
    .loc[lambda df_: pd.to_datetime(df_["From"]).dt.year >= 2020]
    # drop membership code na
    .loc[lambda df_: ~(df_["Membership/Display Name"].isna())]
    # filter cancelled member and free member and non member
    .loc[
        lambda df_: ~(
            df_["Membership Status"].isin(
                ["Cancelled Member", "Free Member", "Non Member"]
            )
        )
    ]
    # filter staff
    .loc[lambda df_: df_["Contract Type"] != "Employee"]
    # rename column
    .rename(
        columns=lambda c: (
            c.lower()
            .replace("/display name", "")
            .replace("/month", "")
            .replace("position", "")
            .strip()
            .replace(" ", "_")
            .replace("/", "_")
        )
    )
    .rename(columns=module.to_rename)
    # drop unused cols
    .drop(columns=module.to_drop)
    .assign(
        # map center name
        center=lambda df_: df_["center"].str.upper().map(module.center_map),
        # clean city
        city=lambda df_: (df_["city"].str.title().str.strip()),
        # clean dob
        dob=lambda df_: np.where(
            df_["dob"].isna(), pd.to_datetime(df_["dob2"]), pd.to_datetime(df_["dob"])
        ),
        # clean start_date, end_date, fp_date
        start_date=lambda df_: pd.to_datetime(df_["start_date"]),
        end_date=lambda df_: pd.to_datetime(df_["end_date"]),
        fp_date=lambda df_: pd.to_datetime(df_["fp_date"]),
        fp_month=lambda df_: df_["fp_date"].dt.strftime("%m / %b %Y"),
        fp_year=lambda df_: df_["fp_date"].dt.strftime("%Y"),
        # clean income
        income=lambda df_: df_["income"].astype(module.income_cat),
        # clean job
        job=lambda df_: np.where(
            df_["job1"].isna(),
            df_["job2"].str.lower().str.replace("[^\w\s]", "", regex=True).str.strip(),
            df_["job1"].str.lower().str.replace("[^\w\s]", "", regex=True).str.strip(),
        ),
        # create age
        age=lambda df_: (
            (df_["start_date"] - df_["dob"])
            .div(pd.Timedelta("365 days"))
            .apply(np.floor)
        ),
        # create id
        id=lambda df_: df_["name"] + " " + df_["dob"].astype(str),
        # create membership code
        membership_code=lambda df_: module.get_membership_code(df_["product"]),
        # create membership duration
        membership_duration=lambda df_: module.get_membership_duration(df_["product"]),
    )
    # merge with membership mapping to obtain membership
    .merge(
        right=pd.read_excel(Path("input/membership_mapping.xlsx")),
        on="membership_code",
        how="left",
    )
    # # make sure that all corporate is mapped
    # .assign(
    #     is_cpt=lambda df_: np.where(
    #         df_["product"].str.lower().str.contains("cpt|corporate|corp", regex=True),
    #         True,
    #         df_["is_cpt"],
    #     )
    # )
    # # ! finish making membership
    # # start adding is_renewal and is_active column
    # # sort by id
    # .sort_values(["id", "end_date"], ascending=[True, False])
    # # ! create renewal column
    # .assign(is_next_contract=lambda df_: df_["id"].duplicated(keep="last"))
    # # ! finish adding is_renewal column
    # # drop unused columns
    # .drop(columns=["dob2", "job1", "job2", "name"])
    # # drop membership code na
    # .loc[lambda df_: ~(df_["membership_code"].isna())]
    # # sort column
    # .sort_index(axis=1)
    # # ! create is_active
    # .assign(
    #     active_jan_2023=lambda df_: module.is_active(
    #         df_, "start_date", "end_date", "1 jan 2023"
    #     ),
    #     active_feb_2023=lambda df_: module.is_active(
    #         df_, "start_date", "end_date", "1 feb 2023"
    #     ),
    #     active_mar_2023=lambda df_: module.is_active(
    #         df_, "start_date", "end_date", "1 mar 2023"
    #     ),
    #     active_apr_2023=lambda df_: module.is_active(
    #         df_, "start_date", "end_date", "1 apr 2023"
    #     ),
    #     active_may_2023=lambda df_: module.is_active(
    #         df_, "start_date", "end_date", "1 may 2023"
    #     ),
    #     active_jun_2023=lambda df_: module.is_active(
    #         df_, "start_date", "end_date", "1 jun 2023"
    #     ),
    #     active_jul_2023=lambda df_: module.is_active(
    #         df_, "start_date", "end_date", "1 jul 2023"
    #     ),
    #     active_aug_2023=lambda df_: module.is_active(
    #         df_, "start_date", "end_date", "1 aug 2023"
    #     ),
    #     active_sep_2023=lambda df_: module.is_active(
    #         df_, "start_date", "end_date", "1 sep 2023"
    #     ),
    #     active_oct_2023=lambda df_: module.is_active(
    #         df_, "start_date", "end_date", "1 oct 2023"
    #     ),
    #     active_nov_2023=lambda df_: module.is_active(
    #         df_, "start_date", "end_date", "1 nov 2023"
    #     ),
    #     active_dec_2023=lambda df_: module.is_active(
    #         df_, "start_date", "end_date", "1 dec 2023"
    #     ),
    # )
)
df_clean[["product", "membership_code", "membership_duration"]].sample(10)

,product,membership_code,membership_duration
8938,[VIP.6] 6 Months Membership VIP - 2022 - PKW,VIP,6
3209,[VIP.6] 6 Months Membership VIP - 2023 - GC,VIP,6
2091,[GO.12] 12 Months Membership GO - 2023 - NST,GO,12
10547,[VIP.ILT.24] 24 Months Membership VIP + IELTS ...,VIP ILT,24
545,[GO.15] Emerald - 15 Months Membership GO - 20...,GO,15
3002,[UP.2.VIP.12] Upgrade Deluxe TO VIP 12 months ...,UP 2 VIP,2
7250,[OLD.CLB.DLX.ILT.15] Bundle Package - 15 Month...,OLD CLB DLX ILT,15
2563,[DLX.15] 15 Months Membership Deluxe - 2023 - PP,DLX,15
9253,[OLD.CLB.VIP.ILT.15] 15 Months Membership VIP ...,OLD CLB VIP ILT,15
3546,[DLX.15] 15 Months Membership Deluxe - 2022 - NST,DLX,15


# Tests and Assertions

In [84]:
# assert that all centers are mapped
center_in_df = df["Partner/Branch/Display Name"].unique()

unmapped = []
for c in center_in_df:
    if isinstance(c, str) and c.upper() not in module.center_map.keys():
        unmapped.append(c)

if len(unmapped) > 0:
    print(unmapped)
    raise Exception("There are unmapped centers. Map this inside modules/center_map.")

In [78]:
# assert that all membership codes has ben accounted
codes = df_clean["membership_code"].unique()
maps = pd.read_excel(Path("input/membership_mapping.xlsx"))["membership_code"].unique()

unmapped = []
for code in codes:
    if code not in maps:
        if code == np.NaN:
            continue
        unmapped.append(code)

if len(unmapped) > 0:
    for i in unmapped:
        print(i)
    raise Exception("Some membership are not mapped.")

# Save DF

In [79]:
import os
file= df_clean
path= f'output/member_data_{date}.parquet'

if not os.path.exists(path):
    file.to_parquet(path)
    print('File saved.')
else:
    print('File already exist.')

File saved.


/home/anj/miniconda3/envs/data-science/lib/python3.11/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


# Experiment

In [80]:
# df.loc[df['Membership/Display Name'].str.lower().str.contains('ilt', na= False)].sort_values('From')

In [81]:
# df['Membership/Display Name'].unique()

In [82]:
# df_clean['is_cpt'].value_counts()